In [8]:
library(dplyr)

In [11]:
# # load full list of results sc stages (iPS, mesendo, defendo)
# res.ips = read.csv("/nfs/leia/research/stegle/acuomo/mean/day0/all_expts/allresults.csv", row.names = 1)
# res.mes = read.csv("/nfs/leia/research/stegle/acuomo/mean/mesendo_est_June20/allresults.csv", row.names = 1)
# res.defendo = read.csv("/nfs/leia/research/stegle/acuomo/mean/defendo_est_June20/allresults.csv", row.names = 1)

In [2]:
# load lead variant results sc stages (iPS, mesendo, defendo)
leads.ips = read.csv("/nfs/leia/research/stegle/acuomo/mean/day0/all_expts/leads.csv", row.names = 1)
leads.mes = read.csv("/nfs/leia/research/stegle/acuomo/mean/mesendo_est_June20/leads.csv", row.names = 1)
leads.defendo = read.csv("/nfs/leia/research/stegle/acuomo/mean/defendo_est_June20/leads.csv", row.names = 1)

In [3]:
leads.ips.sign = leads.ips[leads.ips$global_corr_p_value < 0.1,]
leads.mes.sign = leads.mes[leads.mes$global_corr_p_value < 0.1,]
leads.defendo.sign = leads.defendo[leads.defendo$global_corr_p_value < 0.1,]

In [9]:
RankAndKeepTop <- function(leads){
    leads1 = leads[order(leads$global_corr_p_value),]
    leads2 = leads1[-which(duplicated(leads1$feature)),]  
    leads2
}

In [12]:
### inner join significant sc iPS leads, all results from sc mesendo leads
leads.ips.in.me = left_join(leads.ips.sign, res.mes, by = c("snp_id","feature"), 
                            suffix = c(".ips",".me"))
# nrow(leads.ips.in.me)
leads.ips.in.me = RankAndKeepTop(leads.ips.in.me)
# nrow(leads.ips.in.me)
### inner join significant sc iPS leads and mesenso, all results from sc defendo leads
leads.ips.in.me.in.de = left_join(leads.ips.in.me, res.defendo, by = c("snp_id","feature","ensembl_gene_id"))
# colnames(leads.ips.in.me.in.de)
# nrow(leads.ips.in.me.in.de)
leads.ips.in.me.in.de = RankAndKeepTop(leads.ips.in.me.in.de)
# nrow(leads.ips.in.me.in.de)
ips = leads.ips.in.me.in.de
ips$p_value.de = ips$p_value
ips$beta.de = ips$beta
ips = ips[!(is.na(ips$p_value.me)) & !(is.na(ips$p_value.de)),]
nrow(ips[(ips$p_value.me >= 0.05) & (ips$p_value.de >= 0.05),])

[1] 468

In [13]:
### inner join significant sc mesendo leads, all results from sc iPSC leads
leads.me.in.ips = left_join(leads.mes.sign, res.ips, by = c("snp_id","feature"), 
                            suffix = c(".me",".ips"))
# nrow(leads.me.in.ips)
leads.me.in.ips = RankAndKeepTop(leads.me.in.ips)
nrow(leads.me.in.ips)
# ### inner join significant sc mesendo leads and iPS, all results from sc defendo leads
leads.me.in.ips.in.de = left_join(leads.me.in.ips, res.defendo, by = c("snp_id","feature","ensembl_gene_id"))
# # nrow(leads.me.in.ips.in.de)
leads.me.in.ips.in.de = RankAndKeepTop(leads.me.in.ips.in.de)
# # nrow(leads.me.in.ips.in.de)
mesendo = leads.me.in.ips.in.de
mesendo$p_value.de = mesendo$p_value
mesendo$beta.de = mesendo$beta
mesendo = mesendo[!(is.na(mesendo$p_value.ips)) & !(is.na(mesendo$p_value.de)),]
nrow(mesendo[(mesendo$p_value.ips >= 0.05) & (mesendo$p_value.de >= 0.05),])

[1] 1679

[1] 455

In [14]:
### inner join significant sc defendo leads, all results from sc iPSC leads
leads.de.in.ips = left_join(leads.defendo.sign, res.ips, by = c("snp_id","feature"), 
                            suffix = c(".de",".ips"))
# nrow(leads.de.in.ips)
leads.de.in.ips = RankAndKeepTop(leads.de.in.ips)
nrow(leads.de.in.ips)
# ### inner join significant sc defendo leads and iPS, all results from sc mesendo leads
leads.de.in.ips.in.me = left_join(leads.de.in.ips, res.mes, by = c("snp_id","feature","ensembl_gene_id"))
# # nrow(leads.de.in.ips.in.me)
leads.de.in.ips.in.me = RankAndKeepTop(leads.de.in.ips.in.me)
# # nrow(leads.de.in.ips.in.me)
defendo = leads.de.in.ips.in.me
defendo$p_value.me = defendo$p_value
defendo$beta.me = defendo$beta
defendo = defendo[!(is.na(defendo$p_value.ips)) & !(is.na(defendo$p_value.me)),]
nrow(defendo[(defendo$p_value.ips >= 0.05) & (defendo$p_value.me >= 0.05),])

[1] 1322

[1] 435

In [18]:
ips_specific = ips[(ips$p_value.me >= 0.05) & (ips$p_value.de >= 0.05),]
ips_specific$eqtl = paste0(ips_specific$feature,"-",ips_specific$snp_id)
# head(ips_specific,2)
ips_eqtls = unique(ips_specific$eqtl)
length(ips_eqtls)
head(ips_eqtls,2)

[1] 468

[1] "ENSG00000124562_SNRPC-6_34690778_C_A" 
[2] "ENSG00000211772_TRBC2-7_142450556_G_A"

In [19]:
mesendo_specific = mesendo[(mesendo$p_value.ips >= 0.05) & (mesendo$p_value.de >= 0.05),]
mesendo_specific$eqtl = paste0(mesendo_specific$feature,"-",mesendo_specific$snp_id)
# head(mesendo_specific,2)
mesendo_eqtls = unique(mesendo_specific$eqtl)
length(mesendo_eqtls)
head(mesendo_eqtls,2)

[1] 455

[1] "ENSG00000122550_KLHL7-7_23186699_C_T"   
[2] "ENSG00000089351_GRAMD1A-19_35495012_A_G"

In [26]:
defendo_specific = defendo[(defendo$p_value.ips >= 0.05) & (defendo$p_value.me >= 0.05),]
defendo_specific$eqtl = paste0(defendo_specific$feature,"-",defendo_specific$snp_id)
# head(defendo_specific,2)
defendo_eqtls = unique(defendo_specific$eqtl)
length(defendo_eqtls)
head(defendo_eqtls,2)

[1] 435

[1] "ENSG00000204347_BTBD17-17_72305793_C_T"   
[2] "ENSG00000203805_PPAPDC1A-10_122218993_G_A"

In [51]:
#### 
# df = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/genetic_effect/scaled_absolute_eff_sizes_20mofas_as_spatial_downsample_20pct_cluster_labels.csv", row.names = 1)
df = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/genetic_effect/scaled_eff_sizes_20mofas_as_spatial_downsample_20pct_cluster_labels.csv", row.names = 1)
df$eqtl = gsub("_-","-",df$gene)
head(df,2)

,label,gene,pp0,pp1,pp2,pp3,pp4,pp5,pp6,pp7,pp8,pp9,pp10,pp11,pp12,pp13,pp14,pp15,pp16,eqtl
0,12,ENSG00000005059_CCDC109B_-4_110648632_T_A,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,ENSG00000005059_CCDC109B-4_110648632_T_A
1,6,ENSG00000006016_CRLF1_-19_18735221_G_T,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,ENSG00000006016_CRLF1-19_18735221_G_T


In [52]:
df0 = df[,c("label","eqtl")]
df0$specific = "no"
df0[df0$eqtl %in% ips_eqtls,"specific"] = "ipsc"
df0[df0$eqtl %in% mesendo_eqtls,"specific"] = "mesendo"
df0[df0$eqtl %in% defendo_eqtls,"specific"] = "defendo"
head(df0)

,label,eqtl,specific
0,12,ENSG00000005059_CCDC109B-4_110648632_T_A,mesendo
1,6,ENSG00000006016_CRLF1-19_18735221_G_T,defendo
2,9,ENSG00000006459_KDM7A-7_139913409_G_A,defendo
3,13,ENSG00000013297_CLDN11-3_170402765_T_C,defendo
4,13,ENSG00000021300_PLEKHB1-11_73339784_G_A,no
5,9,ENSG00000029639_TFB1M-6_155635808_C_G,no


In [53]:
table(df0$specific)


defendo    ipsc mesendo      no 
     53      37       6     225 

In [54]:
table(df0$label)


 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 
 8 18 12 35 14  2 13 12  4 59  5  6 28 66  6  2 31 

In [55]:
table(df0$specific, df0$label)

         
           0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16
  defendo  1  0  1 10  2  0  2  4  0 10  0  0  8 12  0  0  3
  ipsc     0  3  1  5  2  0  4  0  0  6  2  1  0  9  2  1  1
  mesendo  0  1  0  0  0  0  0  0  1  0  0  0  2  0  0  1  1
  no       7 14 10 20 10  2  7  8  3 43  3  5 18 45  4  0 26